# Exercise I

## Exploratory Data Analysis (EDA)

In our first exercise, we will explore a public dataset of Coronavirus PCR tests based on a fascinating [blog post](https://blog.mafatchallenge.com/2020/04/30/covid-19-testing-with-ml-part-2/) published as part of the [MAFAT Challenge](https://mafatchallenge.mod.gov.il/).

The purpose of this exercise is to demonstrate the importance of inspecting data and understanding it before trying to do anything fancy. As elementary as it may sound, this preliminary step is the pitfall of many data analysis pipelines and nurturing an instinct for visualizing and organizing your raw data will pay off tremendously both in the process of developing and in evaluating the robustness of your models.

We'll start things off by summarizing some key information about the dataset:

* The dataset is shared as [a GitHub repository](https://github.com/mdcollab/covidclinicaldata) containing a directory of CSV files.
* It is collected by two US-based companies providing services in the health industry.
* The number of observations is still relatively small, but more are added weekly and the quality of the data is high (there are many features to work with and it is relatively organized and "clean").
* There are also chest X-rays available for some of the patients, that were not included in this analysis.

So far so good, let's get to work. 

### Data Retrieval

In order to keep things orderly and easy to maintain, we will first create a file with some general configurations:

````{toggle} configuration.py
    :show:
```{literalinclude} configuration.py   
```
````

And then write a short module to create a `read_data()` method:

````{toggle} read_data.py
    :show:
```{literalinclude} read_data.py
```
````

In [1]:
from read_data import read_data

Now we can read the data by simply running:

In [2]:
data = read_data()

### Raw Inspection

````{admonition} Technical Note: get_scattered_chunks
    :class: dropdown, tip
```{literalinclude} get_scattered_chunks.py
```
````

In [3]:
from get_scattered_chunks import get_scattered_chunks

````{admonition} Technical Note: print_table
    :class: dropdown, tip
We will prepare a function to make the styling of the printed dataframe better suited for our purposes:
```{literalinclude} print_table.py
```
```{tip}
For more information about styling Pandas dataframes, see [the documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html).
```
````

In [4]:
from print_table import print_table

In [5]:
data_chunks = get_scattered_chunks(data, n_chunks=5, chunk_size=3)
print_table(data_chunks)

,batch_date,test_name,swab_type,covid19_test_results,age,high_risk_exposure_occupation,high_risk_interactions,diabetes,chd,htn,cancer,asthma,copd,autoimmune_dis,smoker,temperature,pulse,sys,dia,rr,sats,rapid_flu_results,rapid_strep_results,ctab,labored_respiration,rhonchi,wheezes,days_since_symptom_onset,cough,cough_severity,fever,sob,sob_severity,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,er_referral
0,2020-04-07,SARS COV 2 RNA RTPCR,Nasopharyngeal,False,58,True,nan,False,False,False,False,False,False,False,False,36.950000,81.000000,126.000000,82.000000,18.000000,97.000000,nan,nan,False,False,False,False,28.000000,True,Severe,nan,False,nan,False,False,False,False,False,False,False,False,False
1,2020-04-07,"SARS-CoV-2, NAA",Oropharyngeal,False,35,False,nan,False,False,False,False,False,False,False,False,36.750000,77.000000,131.000000,86.000000,16.000000,98.000000,nan,nan,False,False,False,False,nan,True,Mild,False,False,nan,False,False,False,False,False,False,False,False,False
2,2020-04-07,SARS CoV w/CoV 2 RNA,Oropharyngeal,False,12,nan,nan,False,False,False,False,False,False,False,False,36.950000,74.000000,122.000000,73.000000,17.000000,98.000000,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False
2791,2020-04-28,Rapid COVID-19 Test,Nasopharyngeal,False,56,False,False,False,True,True,False,False,False,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,False,False,nan,False,False,False,False,False,False,False,False,False
2792,2020-04-28,Rapid COVID-19 Test,Nasal,False,73,False,True,False,False,True,False,False,False,True,False,36.750000,86.000000,124.000000,80.000000,16.000000,98.000000,nan,nan,nan,False,nan,nan,5.000000,True,Mild,False,False,nan,False,False,True,False,False,True,True,False,False
2793,2020-04-28,Rapid COVID-19 Test,Nasal,False,25,True,False,False,False,False,False,False,False,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,False,False,nan,False,False,False,False,False,False,False,False,False
5583,2020-05-19,"SARS-CoV-2, NAA",Nasal,False,62,True,False,False,False,False,False,False,False,False,False,37.000000,100.000000,110.000000,76.000000,16.000000,98.000000,nan,nan,False,False,False,False,nan,False,nan,nan,False,nan,True,False,True,False,False,True,True,False,False
5584,2020-05-19,SARS COV2 NAAT,Nasopharyngeal,False,33,False,nan,False,False,False,False,False,False,False,False,36.950000,79.000000,123.000000,79.000000,12.000000,99.000000,nan,nan,nan,False,nan,nan,nan,False,nan,nan,False,nan,False,False,False,False,False,False,False,False,False
5585,2020-05-19,SARS COV2 NAAT,Nasopharyngeal,True,20,True,True,False,False,False,False,False,False,False,False,36.900000,60.000000,114.000000,75.000000,12.000000,97.000000,nan,nan,False,nan,nan,nan,3.000000,True,Mild,True,False,nan,True,False,True,True,True,False,False,False,False
8374,2020-06-09,SARS COV2 NAAT,Nasopharyngeal,False,28,False,False,False,False,False,False,True,False,False,False,36.750000,100.000000,119.000000,85.000000,16.000000,99.000000,nan,nan,False,False,False,False,nan,False,nan,False,False,nan,False,False,False,False,False,False,False,False,False


In [6]:
from configuration import TARGET_COLUMN_NAME
from myst_nb import glue

glue("n_observations", len(data), display=False)
glue("n_columns", len(data.columns), display=False)
glue("target_column_name", TARGET_COLUMN_NAME, display=False)

Alright! Some things that we can already learn about our dataset from this table are:
* It contains a total of {glue:}`n_observations` observations.
* There are {glue:}`n_columns` columns with mixed data types (nemeric and categorical).
* Missing values certainly exist (we can easily spot `nan` entries).
* The subsample raises a strong suspicion that dataset is imbalanced, i.e. when examining our target variable ({glue:}`target_column_name`) it seems there are far more negative observations than positive ones.

There are at least two more built-in methods we could use to get a better sense of the information contained within each column are. The first is:

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11169 entries, 0 to 11168
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   batch_date                     11169 non-null  object 
 1   test_name                      11169 non-null  object 
 2   swab_type                      11169 non-null  object 
 3   covid19_test_results           11169 non-null  bool   
 4   age                            11169 non-null  int64  
 5   high_risk_exposure_occupation  11000 non-null  object 
 6   high_risk_interactions         9668 non-null   object 
 7   diabetes                       11169 non-null  bool   
 8   chd                            11169 non-null  bool   
 9   htn                            11169 non-null  bool   
 10  cancer                         11169 non-null  bool   
 11  asthma                         11169 non-null  bool   
 12  copd                           11169 non-null 

This simple table allows us to easily infer that:
* Most of the collected features are categorial (their data type is `object` or `bool`).
* Some features contain relatively few non-null values.

To select columns by data type we can use the DataFrame object's `select_dtypes()` method:

In [8]:
categorial_columns = data.select_dtypes(["object", "bool"]).columns
numerical_columns = data.select_dtypes(exclude=["object", "bool"]).columns

In [9]:
glue("n_categorical", len(categorial_columns) - 1, display=False)
glue("n_numerical", len(numerical_columns), display=False)

This way we can determine that {glue:}`n_numerical` of the collected features are numerical and {glue:}`n_categorical` are categorical.

Another useful method we can use is:

In [10]:
data.describe()

,age,temperature,pulse,sys,dia,rr,sats,days_since_symptom_onset
count,11169.000000,6542.000000,6525.000000,6551.000000,6551.000000,5762.000000,6386.000000,3573.000000
mean,42.178172,36.875268,78.730115,126.043505,79.706457,15.049462,98.108988,10.142177
std,16.107779,0.316541,13.823534,16.789703,9.865094,2.092585,1.446847,19.023191
min,-3.000000,34.950000,40.000000,75.000000,36.000000,10.000000,76.000000,1.000000
25%,30.000000,36.700000,69.000000,115.000000,74.000000,14.000000,97.000000,2.000000
50%,40.000000,36.850000,78.000000,124.000000,80.000000,16.000000,98.000000,4.000000
75%,54.000000,37.000000,87.000000,135.000000,85.000000,16.000000,99.000000,10.000000
max,90.000000,39.400000,150.000000,235.000000,135.000000,32.000000,100.000000,300.000000


In [11]:
from print_table import get_table_styles

table_styles = get_table_styles()
data.describe().style.set_table_styles(table_styles).set_properties(
    **{"text-align": 'center'})

,age,temperature,pulse,sys,dia,rr,sats,days_since_symptom_onset
count,11169.000000,6542.000000,6525.000000,6551.000000,6551.000000,5762.000000,6386.000000,3573.000000
mean,42.178172,36.875268,78.730115,126.043505,79.706457,15.049462,98.108988,10.142177
std,16.107779,0.316541,13.823534,16.789703,9.865094,2.092585,1.446847,19.023191
min,-3.000000,34.950000,40.000000,75.000000,36.000000,10.000000,76.000000,1.000000
25%,30.000000,36.700000,69.000000,115.000000,74.000000,14.000000,97.000000,2.000000
50%,40.000000,36.850000,78.000000,124.000000,80.000000,16.000000,98.000000,4.000000
75%,54.000000,37.000000,87.000000,135.000000,85.000000,16.000000,99.000000,10.000000
max,90.000000,39.400000,150.000000,235.000000,135.000000,32.000000,100.000000,300.000000


This table summarizes the basic properties of each of the numerical columns contained in the dataset and gives a brief overview of the observed variance.

### Missing Values

Handling missing values requires careful judgement. Possible solutions include:
* Removing the entire feature (column) containing the missing values.
* Removing all observations with missing values.
* "Filling in" missing values with some constant or statistic, such as the mean or the mode. 

There is not one correct method, as different circumstances call for different approaches. If a column contains a small number of observations (relative to the size of the dataset) and our dataset is rich enough and offers more features that could be expected to be informative, we might decide to remove it. Or, if we have a large dataset and the feature in question is crucial for the purposes of our analysis, we might decide to remove all rows with a missing value. Finally, Filling in missing values might be a good trade-off if we have good reason to believe some statistic may adequately approximate the missing values. 

Let's first get a better sense of what fraction of the values is missing in each of the relevant columns:

````{toggle}
    :show:
```{literalinclude} missing_values.py
    :pyobject: calculate_nan_fractions
```
````

In [12]:
from missing_values import calculate_nan_fractions

In [13]:
missing_fractions_df = calculate_nan_fractions(data, target_column=TARGET_COLUMN_NAME)
missing_fractions_df.style.format("{:,.2%}")

,Total Missing,Negatives Fraction,Positives Fraction
Column Name,,,
rapid_strep_results,99.23%,97.20%,2.80%
rapid_flu_results,98.52%,97.26%,2.74%
sob_severity,91.50%,97.24%,2.76%
cough_severity,85.12%,97.69%,2.31%
days_since_symptom_onset,68.01%,97.72%,2.28%
rhonchi,59.61%,97.93%,2.07%
wheezes,56.80%,98.03%,1.97%
rr,48.41%,98.02%,1.98%
ctab,47.39%,98.34%,1.66%


In [14]:
positives_fraction = "{:,.2%}".format(len(data[data[TARGET_COLUMN_NAME]]) / len(data))
mean_positives_null_fraction = "{:,.2%}".format(missing_fractions_df["Positives Fraction"].mean())

glue("mean_positives_null_fraction", mean_positives_null_fraction, display=False)
glue("positives_fraction", positives_fraction, display=False)

This table allows us to easily see which columns containt a large fraction of missing values, but also shows us that the mean relative fraction of missing values in Coronavirus-positive observations ({glue:}`mean_positives_null_fraction`) isn't far from the general fraction of positives ({glue:}`positives_fraction`). This is very good sign of the quality of the data, and assures us the dataset does not include a ["missing not at random"](https://en.wikipedia.org/wiki/Missing_data#Missing_not_at_random) bias.

In [15]:
from configuration import NAN_FRACTION_THRESHOLD

glue("nan_fraction_threshold", "{:,.2%}".format(NAN_FRACTION_THRESHOLD), display=False)

Now, we will create a simple function to remove all columns with more than {glue:}`nan_fraction_threshold` of missing values:

````{toggle}
    :show:
```{literalinclude} missing_values.py
    :pyobject: remove_missing_data_columns
```
````

In [16]:
from missing_values import remove_missing_data_columns

In [17]:
data = remove_missing_data_columns(data)
data.shape

(11169, 25)

For the rest of the columns (with a fraction of missing values that is lower than the threshold), we will simply remove observations with missing values from the dataset:

In [18]:
data = data.dropna(axis=0, how="any").reset_index(drop=True)
data.shape

(10959, 25)

Our dataset is now clean from any missing values and ready for further inspection.

### Data Leakage

Now that we have a basic understanding of the dataset, we should consider whether all of the features are indeed informative and make sure we get rid of any noise or potential data leakage.

```{important}
> "In statistics and machine learning, **leakage** (also **data leakage**, or **target leakage**)
> is the use of information in the model training process which would not be expected
> to be available at prediction time, causing the predictive scores (metrics) to
> overestimate the model's utility when run in a production environment." {cite}`data_leakage`
```

```{seealso}
* [Wikipedia](https://en.wikipedia.org/wiki/Leakage_(machine_learning))
* [Kaggle tutorial](https://www.kaggle.com/alexisbcook/data-leakage)
```

So, what predictors are we left with?

In [23]:
print("Categorial features:\n" + ", ".join(data.select_dtypes(["object"]).columns))
print("\nNumerical features:\n" + ", ".join(data.select_dtypes(exclude=["object"]).columns))

Categorial features:
batch_date, test_name, swab_type, high_risk_exposure_occupation, cough, sob, diarrhea, fatigue, headache, loss_of_smell, loss_of_taste, runny_nose, muscle_sore, sore_throat

Numerical features:
covid19_test_results, age, diabetes, chd, htn, cancer, asthma, copd, autoimmune_dis, smoker, er_referral
